In [3]:
# Import json,requests,pandas,time and csv
import json
import requests
import pandas 
import time
import csv
#Reading csv file  
result= pandas.read_csv("md5.csv") 
#Writing the dataset.csv file 
with open('test.csv', mode='a') as datasetfile:
    dataset_writer = csv.writer(datasetfile,delimiter='|',
                                quotechar='"', quoting=csv.QUOTE_MINIMAL)
    dataset_writer.writerow(['Name', 'md5', 'Machine', 'SectionsMeanEntropy',
       'SectionsMinEntropy', 'SectionsMaxEntropy', 'SectionsMeanRawsize',
       'SectionsMinRawsize', 'SectionMaxRawsize', 'SectionsMeanVirtualsize',
       'SectionsMinVirtualsize', 'SectionMaxVirtualsize','ResourceNb',
       'ResourcesMeanEntropy', 'ResourcesMinEntropy', 'ResourcesMaxEntropy',
       'VersionInformationSize', 'legitimate'])
    #looping through the md5 hash
    for row in range(0,70536):

        #getting the md5 hash cell
        cell=result.loc[row][0]
        #using the Virustotal API to lookup the latest report
        url = 'https://www.virustotal.com/vtapi/v2/file/report'
        params = {'apikey': 
                  'apikey-value',
                  'resource': cell,'allinfo':'true'}
        #Using Dicts from Json reponse to info from the report
        response = requests.get(url, params=params)
        #print(response.json().keys())
        #Getting the File submitted name on Virustotal
        submittedName = response.json().get('submission_names')
        #print(submittedName[0])
        #Getting the addtionaal info for sections entropy
        additionalinfo = response.json().get('additional_info')
        #print(additionalinfo)
        #Printing the resources languages counter
        #print(additionalinfo.get('pe-resource-langs'))
        resourceLang= additionalinfo.get('pe-resource-langs')
        #Resource Number list
        resourceNumber=[]
        if resourceLang!=None:
            #Looping through the resource Language Dictionary
            for lang in resourceLang:
                #print(resourceLang.get(lang))
                resourceNumber.append(resourceLang.get(lang))
        #printing the dictionary keys for additional info page
        #print(additionalinfo.keys())
        #Getting the machine information
        machine =additionalinfo.get('pe-machine-type')
        #Getting the version information size
        versionInfosize=additionalinfo.get('exiftool')
        #Getting the pe resource detail for resource entropy
        peresourcedetail=additionalinfo.get('pe-resource-detail')
        #Getting the section detail for section entropy
        sections = additionalinfo.get('sections')
        #resource,rawsize, virtualsize and section lists to compute max, min and mean
        sectionVirtualSize=[]
        sectionRawSize=[]
        sectionEntropy= []
        resourceEntropy= []
        versionInfosize=additionalinfo.get('exiftool')
        #print(versionInfosize)
        #print(type(versionInfosize))
        #checking the submittedName list is not empty
        if len(submittedName)>0:
            name = submittedName[0]
        #Checking is the resource info and  section info exists in the report
        if (peresourcedetail!=None) and (sections!=None):
            #looping through the section list to get the virtualsize at index 2
            for virtualsize in sections:
                if float(virtualsize[2])!=None:
                    sectionVirtualSize.append(float(virtualsize[2]))
            #looping through the section list to get the rawsize at index 3
            for rawsize in sections:
                if float(rawsize[3])!=None:
                    sectionRawSize.append(float(rawsize[3]))
            #looping through the peresource dict
            for resource in peresourcedetail:
                #checking if getting resource entropy exists 
                if resource.get('entropy')!=None:
                    resourceEntropy.append(resource.get('entropy'))
            #looping through the sections list
            for entropy in sections:
                #Checking if getting section entropy exists
                if float(entropy[4])!=None:
                    sectionEntropy.append(float(entropy[4]))
        legitimate=0
        #checking for a Legitimate files
        if response.json().get('positives')>0:
            legitimate=0
        else:
            legitimate=1
        #checking for NoneType error on the versionInfosize
        if versionInfosize!=None:
            versionInfosize= len(versionInfosize.keys())
            #checking is the sectionVirtualSize,sectionRawSize,sectionEntropy and resourceEntropy lists are not empty       
            if len(sectionVirtualSize)!=0 and len(sectionRawSize)!=0 and len(sectionEntropy)!=0 and len(resourceEntropy)!=0 and len(resourceNumber)!=0:
                #Writing the hash,maxresourceEntropy,minresourceEntropy, 
                #maxsectionEntropy,minsectionEntropy,meansectionEntropy to the dataset
                dataset_writer.writerow([name,cell,machine,format((sum(sectionEntropy)/len(sectionEntropy)),".9f"),
                                         min(sectionEntropy), 
                                         max(sectionEntropy),format((sum(sectionRawSize)/len(sectionRawSize)),".9f")
                                         ,min(sectionRawSize), 
                                         max(sectionRawSize),format((sum(sectionVirtualSize)/len(sectionVirtualSize)),".9f")
                                         ,min(sectionVirtualSize), 
                                         max(sectionVirtualSize),sum(resourceNumber),format((sum(resourceEntropy)/len(resourceEntropy)),".9f")
                                         ,min(resourceEntropy),max(resourceEntropy),versionInfosize,legitimate])